https://deap.readthedocs.io/en/master/overview.html

Create Types

In [1]:
# create types

from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

initalization of individual and population

In [2]:
# initialization

import random
from deap import tools

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

operators: mate, mutate, select, evaluate

In [7]:
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

algorithms

In [8]:
pop = toolbox.population(50)
print(type(pop), len(pop), type(pop[0]), len(pop[0]))
print(pop)

<class 'list'> 50 <class 'deap.creator.Individual'> 10
[[0.42623472431433174, 0.11323225750397625, 0.3531892002394339, 0.8780767698852178, 0.3619788544418402, 0.04182169926121526, 0.9181811615638887, 0.8875499180680865, 0.23852844640129978, 0.9919608286771705], [0.9410759592485787, 0.3086934554163756, 0.5239694614641356, 0.9917751508230388, 0.2985473436301407, 0.951488330122114, 0.2538205175012457, 0.5218080428331323, 0.45818674307732277, 0.7189688364067879], [0.212697637011662, 0.2815238893628823, 0.5696230909269837, 0.4451211156937338, 0.9037186617704754, 0.11000231323733767, 0.7526203864393662, 0.08751091988528537, 0.1778727222437645, 0.2613024418251656], [0.32895010232282007, 0.2535446012226573, 0.8276341760580189, 0.9523826672584146, 0.3903647264488388, 0.6298313523715882, 0.47808032308818094, 0.8242547488731186, 0.5859868434848899, 0.7982008133425834], [0.23774924313595303, 0.40124668010278364, 0.16625840287522164, 0.019499624869501075, 0.304822347218916, 0.24042973957125413, 0.4

In [13]:
CXPB, MUTPB, NGEN = 0.5, 0.2, 40

fitnesses = map(toolbox.evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

for g in range(NGEN):
    offspring = toolbox.select(pop, len(pop))
    print("offspring", type(offspring), len(offspring))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    pop[:] = offspring
    

print(pop)

offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'list'> 50
offspring <class 'li

In [ ]:
次は、コードの理解から